<a href="https://colab.research.google.com/github/ManuelOtt-code/Project-DC/blob/master/03b_Mordred_Feature_Calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Due to time and computational reasons, Mordred features were calculated in a seperate code here and were saved manually to be used in 03b_Mordred_ML_Training file. You do not need to run this code to generate the data and can directly go to 03b_Mordred_ML_Training.

In [ ]:
# Install packages
!pip install mordred
!pip install numpy==1.23.5
!pip install rdkit
from mordred import Calculator, descriptors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 29.8 MB/s eta 0:00:00


In [ ]:
# Get data that we previously currated or you can skip this if you have it already downloaded to your google drive or wherever
# you run this
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master/func.py
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_Phenol_training_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_Phenol_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_All_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_All_training_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_Phenol_training_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_Phenol_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_All_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_All_training_data.csv

--2025-06-30 07:45:33--  https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master/func.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ManuelOtt-code/Project-DC/refs/heads/master/func.py [following]
--2025-06-30 07:45:33--  https://raw.githubusercontent.com/ManuelOtt-code/Project-DC/refs/heads/master/func.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6467 (6.3K) [text/plain]
Saving to: ‘func.py’

func.py             100%[===================>]   6.32K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2025-06-30 07:45:33 (85.9 MB/s) - ‘func.py’ sav

In [ ]:
# all CSV files -> dataframe
import pandas as pd

df_Landrum_Phenol_training_data = pd.read_csv("Landrum_Phenol_training_data.csv")
df_Landrum_Phenol_test_data = pd.read_csv("Landrum_Phenol_test_data.csv")
df_Landrum_All_training_data = pd.read_csv("Landrum_All_training_data.csv")
df_Landrum_All_test_data = pd.read_csv("Landrum_All_test_data.csv")
df_Minimal_Phenol_training_data = pd.read_csv("Minimal_Phenol_training_data.csv")
df_Minimal_Phenol_test_data = pd.read_csv("Minimal_Phenol_test_data.csv")
df_Minimal_All_training_data = pd.read_csv("Minimal_All_training_data.csv")
df_Minimal_All_test_data = pd.read_csv("Minimal_All_test_data.csv")

In [ ]:
# Upload dataframes and choose the subset of data that the mordred features are going to be generated for

datasets = {
    "Landrum_Phenol_training": df_Landrum_Phenol_training_data,
    "Landrum_All_training": df_Landrum_All_training_data,
    "Landrum_Phenol_test": df_Landrum_Phenol_test_data,
    "Landrum_All_test": df_Landrum_All_test_data,
    "Minimal_Phenol_training": df_Minimal_Phenol_training_data,
    "Minimal_All_training": df_Minimal_All_training_data,
    "Minimal_Phenol_test": df_Minimal_Phenol_test_data,
    "Minimal_All_test": df_Minimal_All_test_data,
}

COMPUTE = {
    "Landrum_Phenol_training": False,
    "Landrum_All_training": True,
    "Landrum_Phenol_test": False,
    "Landrum_All_test": True,
    "Minimal_Phenol_training": False,
    "Minimal_All_training": False,
    "Minimal_Phenol_test": False,
    "Minimal_All_test": False,
}

In [ ]:
# prompt: provide a function to calculate all 2D descriptors from mordred
# using the smiles from the df dataframe and append the calculated features to the same dataframe


def generate_mordred_descriptors(df):
    """Calculates all 2D descriptors from Mordred and appends them to the DataFrame.

    Args:
        df: Input DataFrame with a 'canonical_Smiles' column.

    Returns:
        DataFrame with added Mordred descriptors.
    """

    calc = Calculator(
        descriptors, ignore_3D=True
    )  # Initialize Mordred calculator (2D only)

    # Calculate descriptors for valid molecules
    def calculate_mordred_descriptors_for_mol(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return calc(mol)
        else:
            return None  # Return None for invalid SMILES

    df["mordred_descriptors"] = df["canonical_Smiles"].apply(
        calculate_mordred_descriptors_for_mol
    )

    # Expand the Mordred descriptor dictionary into separate columns
    mordred_df = pd.DataFrame(df["mordred_descriptors"].tolist())

    # Concatenate the expanded features with the original DataFrame
    df = pd.concat([df, mordred_df], axis=1)

    # Drop the original mordred_descriptors column
    df = df.drop("mordred_descriptors", axis=1)

    return df

In [ ]:
# Generate mordred features
from rdkit import Chem

mordred_descriptors = {}

for name, df in datasets.items():
    if COMPUTE.get(name, False):
        print(f"✅ Computing: {name}")
        mordred_descriptors[name] = generate_mordred_descriptors(df)
        mordred_descriptors[name].to_csv(f"{name}_mordred.csv", index=False)
    else:
        print(f"⏭️ Skipping: {name}")

⏭️ Skipping: Landrum_Phenol_training
✅ Computing: Landrum_All_training


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  


⏭️ Skipping: Landrum_Phenol_test
✅ Computing: Landrum_All_test


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  


⏭️ Skipping: Minimal_Phenol_training
⏭️ Skipping: Minimal_All_training
⏭️ Skipping: Minimal_Phenol_test
⏭️ Skipping: Minimal_All_test
